In [1]:
#Importing all the necessarily libraries

import heartpy as hp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Importing the dataset

dataset = pd.read_csv('ECG_training.csv')

In [3]:
#ecgdata contains all the input data from the dataset
ecgdata = []

#Y is the classification column for the dataset
Y = []

In [4]:
ecg_reading = dataset['ECG']

In [5]:
Y = dataset['Classification']

In [6]:
#getting all the ecg data from the dataset to ecgdata

for i in range(len(ecg_reading)):
    ecg = ecg_reading[i]
    ecg = ecg.split(',')
    ecg = [int(j) for j in ecg]
    ecgdata.append(ecg)

In [7]:
# len(ecgdata) # = 3500

In [8]:
# making dataframe for the features to be extracted frgom the ecg data

X = pd.DataFrame()

In [9]:
for i in range(len(ecgdata)):

    
    k = [120,121,139,604,825,1112,1355,1847,1975,2071,2103,2305,2306,2448,2504,2599,2640,2981,3201,3325] 
            # Rows of data where the noise is too high and their exclusion improves the accuracy
    if(i in k):
        continue
    
    
    ecg = ecgdata[i]
    ecg = np.nan_to_num(ecg)
    working_data, measures = hp.process(ecg, 300.0)
    df_dictionary = pd.DataFrame([measures])
    X = pd.concat([X, df_dictionary], ignore_index=True)


C:\Users\prana\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:279: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
C:\Users\prana\anaconda3\lib\site-packages\numpy\ma\core.py:5243: RuntimeWarning: Mean of empty slice.
  result = super(MaskedArray, self).mean(axis=axis,
C:\Users\prana\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3621: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\prana\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\prana\anaconda3\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarni

In [10]:
#The extracted features

X

,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate
0,78.774617,761.666667,61.300477,83.718564,93.857921,0.459459,0.108108,28.333333,66.365096,57.024892,11889.239379,1.163792,0.241963
1,363.636364,165.000000,114.646801,102.282345,155.512466,0.857143,0.714286,66.666667,106.441088,110.212637,36854.504128,0.965779,0.303674
2,64.425412,931.309524,169.711097,133.625235,219.979603,0.923077,0.807692,135.000000,155.330156,183.697974,89641.686245,0.845574,0.191806
3,151.840491,395.151515,179.203421,142.993395,360.872400,1.000000,1.000000,140.000000,254.442310,130.290274,104148.063640,1.952888,0.230256
4,62.666821,957.444444,51.766451,20.782041,29.980837,0.413793,0.103448,45.000000,20.973468,69.332834,4568.346982,0.302504,0.139315
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475,100.352113,597.894737,131.390274,142.412243,185.017517,0.733333,0.400000,43.333333,130.648454,137.846511,56578.303967,0.947782,0.176157
3476,151.431441,396.218905,142.636738,27.960504,277.541046,1.000000,1.000000,30.000000,196.212843,48.645606,29986.164465,4.033516,0.226202
3477,192.051214,312.416667,205.838377,174.553001,326.424230,0.964286,0.892857,191.666667,229.513235,150.216091,108311.373339,1.527887,0.240327
3478,82.711086,725.416667,45.957030,8.822898,14.935760,0.128205,0.000000,21.666667,10.498566,64.008706,2111.149057,0.164018,0.172408


In [11]:
# Removing the same rows from the classification column

k = [120,121,139,604,825,1112,1355,1847,1975,2071,2103,2305,2306,2448,2504,2599,2640,2981,3201,3325]
for i in k:
    Y.pop(i)

In [12]:
X['Classification'] = Y
X.to_csv('Extracted_Features.csv',index=False)
X = X.drop('Classification',axis=1)

In [13]:
# Removing the Nan 

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(X)

X = imputer.transform(X)


C:\Users\prana\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)
C:\Users\prana\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)


In [14]:

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.15)

################################################

In [15]:
#Using Sklearn Random Forest Classifier for Binary Classification

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
predictions = rf.predict(X_test)

from sklearn.metrics import f1_score
print(f1_score(y_test,predictions,pos_label='N'))

0.9612068965517241
